In [22]:
import json

# Load data
with open("/home/broodling/finQA/pre_values_0928.json", "r") as f5:
  preV = json.load(f5)

with open("/home/broodling/finQA/table_values_0928.json", "r") as f6:
  tableV = json.load(f6)

with open("/home/broodling/finQA/post_values_0928.json", "r") as f7:
  postV = json.load(f7)

with open("/home/broodling/finQA/datasets/FinQA/dataset/train.json", "r") as f2:
  golds = json.load(f2)

check = []
needs = []
for gold in golds[:200]:
  need = gold["qa"]["gold_inds"]
  needs.append(need)
  check.append(gold["qa"]["steps"])

print(len(tableV), len(needs))

retri =[]
tpre =""
ttable=""
tpost=""
for pre, table, post in zip(preV, tableV, postV):
  for i in range(0, len(pre)):
    tpre = tpre+pre[i]
  for j in range(0, len(table)):
    ttable = ttable+table[j]
  for k in range(0, len(post)):
    tpost = tpost+post[k]
  total = tpre + "\n" + ttable + "\n" + tpost
  retri.append(total)

print(len(retri))

200 200
200


In [44]:
import json

# Load data
with open("/home/broodling/finQA/retrieved_info_1002_200.json", "r") as f:
  extracts = json.load(f)

with open("/home/broodling/finQA/datasets/FinQA/dataset/train.json", "r") as f2:
  golds = json.load(f2)

needs = []
for gold in golds[:200]:
  need = gold["qa"]["gold_inds"]
  needs.append(need)

schemas = []
values =[]
c=0
for extr in extracts:
  try:
    sch, vals = extr.split("Values")
    sch = sch.split("Extracted Schema")[1]
  except:
    if c==31:
      sch = extr.split("```sql")[1]
      sch = sch.split("```")[0]
      vals = sch
    else:
      sch = extr
      vals = extr
  # sch = sch.split("Extracted Schema")[1]
  sch = sch.lstrip(":")
  vals = vals.lstrip(":")
  sch = sch.lstrip("\n")
  sch = sch.rstrip("\n")
  vals = vals.lstrip("\n")
  vals = vals.rstrip("\n")
  schemas.append(sch)
  values.append(vals)
  c += 1

In [47]:
# preprocess for evaluate -> only number extracted
import re

def extract_numbers(text):
    numbers = re.findall(r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b|\b\d+\b(?:\.\d+)?', text)
    nums = [float(num.replace(',', '')) if '.' in num else int(num.replace(',', '')) for num in numbers]
    nums = list(set(nums))
    
    return nums

sub_num = []
gold_num = []

for sub in values:
    nums = extract_numbers(sub)
    sub_num.append(nums)
    #print(nums)

for need in needs:
    inds= list(need.values())
    st = ""
    for ind in inds:
        st += ind+" "
    nums = extract_numbers(st)
    gold_num.append(nums)
    #print(nums)

In [48]:
# precision, f1, recall
def calculate_custom_metrics(predictions, actuals):
    tp = len(set(actuals) & set(predictions))   # True Positives (TP)
    fp = len(set(predictions) - set(actuals))   # False Positives (FP)
    fn = len(set(actuals) - set(predictions))   # False Negatives (FN)
    
    # Precision, Recall, F1 Score, Accuracy 계산
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = tp / len(actuals) if len(actuals) > 0 else 0 # recall과 같은 결과
    
    return precision, recall, f1, accuracy

pr=[]
rec=[]
f =[]
acc = []

# Calculate Metric
for sub, gol in zip(sub_num, gold_num):
    precision, recall, f1, accuracy = calculate_custom_metrics(sub, gol)
    pr.append(precision)
    rec.append(recall)
    f.append(f1)
    acc.append(accuracy)


print(len(pr), len(rec), len(f), len(acc))
print(f"Precision: {round(sum(pr)/len(pr),3)}")
print(f"Recall: {round(sum(rec)/len(rec), 3)}")
print(f"F1 Score: {round(sum(f)/len(f),3)}")
print(f"Accuracy: {round(sum(acc)/len(acc),3)}")

200 200 200 200
Precision: 0.492
Recall: 0.528
F1 Score: 0.442
Accuracy: 0.528


In [4]:
with open("/home/broodling/finQA/pre_table_0817.json", "r") as f3:
  pres = json.load(f3)

with open("/home/broodling/finQA/post_table_0817.json", "r") as f4:
  posts = json.load(f4)

pre_num = []
post_num = []

for pre, post in zip(pres, posts):
  prenum = extract_numbers(pre)
  postnum = extract_numbers(post)
  pre_num.append(prenum)
  post_num.append(postnum)


print(len(pre_num), len(post_num))
print(pre_num[4])
print(post_num[4])

6251 6251
[2008, 2007]
[56.8, 364.7, 2007]


In [5]:
# Calculate total metric
gold_nums = gold_num
pred_nums = []
for pre, pos, sub in zip(pre_num, post_num, sub_num):
    pred_num = list(set(pre+pos+sub))
    pred_nums.append(pred_num)

# print(len(pred_nums), len(gold_nums))

prt=[]
rect=[]
ft =[]
acct = []

for sub, gol in zip(pred_nums[1:], gold_nums[1:]):
    precision, recall, f1, accuracy = calculate_custom_metrics(sub, gol)
    prt.append(precision)
    rect.append(recall)
    ft.append(f1)
    acct.append(accuracy)

print(len(prt), len(rect), len(ft), len(acct))
print(f"Precision: {round(sum(prt)/len(prt), 3)}")
print(f"Recall: {round(sum(rect)/len(rect), 3)}")
print(f"F1 Score: {round(sum(ft)/len(ft), 3)}")
print(f"Accuracy: {round(sum(acct)/len(acct), 3)}")

6250 6250 6250 6250
Precision: 0.314
Recall: 0.799
F1 Score: 0.417
Accuracy: 0.799


In [10]:
import json 

with open("/home/broodling/finQA/datasets/FinQA/dataset/train.json", "r") as f:
  golds = json.load(f)

answers = []
for gold in golds:
  ans = gold['qa']['exe_ans']
  answers.append(ans)

# with open("/home/broodling/finQA/final_answer_0929_200.json", "r") as f2:
#   executes = json.load(f2)

with open("/home/broodling/finQA/final_answer_1004_tools.json", "r") as f3:
  tools = json.load(f3)

In [11]:
executes =[]
functions = {"sql":0, "equation":0, "code":0}

for tool in tools:
  answer=tool.split("Answer: ")[1]
  func = tool.split("Function Content:")[0]
  executes.append(answer)
  if 'execute_sqlite3' in func:
    functions['sql'] += 1
  elif 'equation_' in func:
    functions['equation'] += 1
  else:
    functions["code"] += 1

In [19]:
# 0929 200개 test => 0.01% 정답률
correct = 0
for gold, exe in zip(answers, executes):
  try:
    ans = round(float(gold),4)
  except:
    ans = gold

  try:
    pred = round(float(exe),4)
  except:
    if ("No" in exe) or ("no" in exe) or ("Yes" in exe) or ("yes" in exe):
      pred = exe
    else:
      pred = 0

  # check answer
  # print(ans, pred)
  if ans==pred:
    correct += 1
  elif (type(pred)==float) and (pred*100==gold):
    correct += 1
  elif (type(pred)==float) and (gold*100==pred):
    correct += 1

print(correct/200*100) # 2 -> 13(6.5%)

6.5
